# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [253]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Ask for parameters

In [254]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [255]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**1. Binary variables for pipe activation (1 if active, 0 if inactive):**

*  x_AB
*  x_AC
*  x_AF
*  x_BC
*  x_BD
*  x_BE
*  x_CE
*  x_CF
*  x_DE
*  x_EG
*  x_FB
*  x_FG

**2. Continuous variables for water flow through each pipe at each time step (t = 1, 2,..., 10):**

* f_AB(t)
* f_AC(t)
* f_AF(t)
* f_BC(t)
* f_BD(t)
* f_BE(t)
* f_CE(t)
* f_CF(t)
* f_DE(t)
* f_EG(t)
* f_FB(t)
* f_FG(t)

**3. Continuous variables for water released at each point at each time step (t=1, 2,..., 10):**

* r_B(t)
* r_C(t)
* r_D(t)
* r_E(t)
* r_F(t) 


# 2. Ask for objective

In [256]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [257]:
Markdown(response2.text)

Maximize:  ∑_{t=1}^{10} (f_EG(t) + f_FG(t)) 


# 3. Ask for constraints

In [258]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [259]:
Markdown(response3.text)

## Constraints:

**1. Pipe Activation:**

*   **Limit on active pipes:**  ∑_{i∈{AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}} x_i = 10 

**2. Water Flow at each Time Step (t = 1, 2, ..., 10):**

*   **Inflow at A:**  f_AB(t) + f_AC(t) + f_AF(t)  = f(t) 
*   **Conservation of flow at B:** f_AB(t) + f_FB(t) = f_BC(t) + f_BD(t) + f_BE(t) + r_B(t)
*   **Conservation of flow at C:** f_AC(t) + f_BC(t) = f_CE(t) + f_CF(t) + r_C(t)
*   **Conservation of flow at D:** f_BD(t) + f_DE(t) = r_D(t) 
*   **Conservation of flow at E:** f_BE(t) + f_CE(t) + f_DE(t) = f_EG(t) + r_E(t)
*   **Conservation of flow at F:** f_AF(t) + f_CF(t) + f_FB(t) = f_FG(t) + r_F(t)

**3. Pipe Capacity:**

*   **Flow capacity limits:**
    *   0 ≤ f_AB(t) ≤ x_AB * 3 
    *   0 ≤ f_AC(t) ≤ x_AC * 6
    *   0 ≤ f_AF(t) ≤ x_AF * 1
    *   0 ≤ f_BC(t) ≤ x_BC * 3
    *   0 ≤ f_BD(t) ≤ x_BD * 10
    *   0 ≤ f_BE(t) ≤ x_BE * 4
    *   0 ≤ f_CE(t) ≤ x_CE * 4
    *   0 ≤ f_CF(t) ≤ x_CF * 4
    *   0 ≤ f_DE(t) ≤ x_DE * 5
    *   0 ≤ f_EG(t) ≤ x_EG * 5
    *   0 ≤ f_FB(t) ≤ x_FB * 12
    *   0 ≤ f_FG(t) ≤ x_FG * 7

**4. Non-negativity of Release:**

* r_B(t) ≥ 0
* r_C(t) ≥ 0
* r_D(t) ≥ 0
* r_E(t) ≥ 0
* r_F(t) ≥ 0 


## 3. Generate the pyomo code

In [260]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [261]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You'll need to replace this with your actual data)
f_data = {1: 10, 2: 12, 3: 15, 4: 10, 5: 8, 6: 12, 7: 14, 8: 11, 9: 9, 10: 13} 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10) # Time periods
model.Pipes = pyo.Set(initialize=["AB", "AC", "AF", "BC", "BD", "BE", "CE", "CF", "DE", "EG", "FB", "FG"])
model.Nodes = pyo.Set(initialize=["B", "C", "D", "E", "F"])

# Variables
model.x = pyo.Var(model.Pipes, domain=pyo.Binary) # Pipe activation
model.f = pyo.Var(model.Pipes, model.T, domain=pyo.NonNegativeReals) # Flow in pipes
model.r = pyo.Var(model.Nodes, model.T, domain=pyo.NonNegativeReals) # Release at nodes

# Objective function
model.obj = pyo.Objective(expr=sum(model.f["EG", t] + model.f["FG", t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Pipe Activation Limit
model.pipe_activation = pyo.Constraint(expr=sum(model.x[i] for i in model.Pipes) == 10)

# 2. Water Flow at each Time Step
def flow_balance(model, t):
    if t==1:
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
    else: 
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
model.flow_A = pyo.Constraint(model.T, rule=flow_balance)

def flow_balance_B(model, t):
    return model.f["AB", t] + model.f["FB", t] == model.f["BC", t] + model.f["BD", t] + model.f["BE", t] + model.r["B", t]
model.flow_B = pyo.Constraint(model.T, rule=flow_balance_B)

def flow_balance_C(model, t):
    return model.f["AC", t] + model.f["BC", t] == model.f["CE", t] + model.f["CF", t] + model.r["C", t]
model.flow_C = pyo.Constraint(model.T, rule=flow_balance_C)

def flow_balance_D(model, t):
    return model.f["BD", t] + model.f["DE", t] == model.r["D", t]
model.flow_D = pyo.Constraint(model.T, rule=flow_balance_D)

def flow_balance_E(model, t):
    return model.f["BE", t] + model.f["CE", t] + model.f["DE", t] == model.f["EG", t] + model.r["E", t]
model.flow_E = pyo.Constraint(model.T, rule=flow_balance_E)

def flow_balance_F(model, t):
    return model.f["AF", t] + model.f["CF", t] + model.f["FB", t] == model.f["FG", t] + model.r["F", t]
model.flow_F = pyo.Constraint(model.T, rule=flow_balance_F)


# 3. Pipe Capacity Limits
model.capacity_AB = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AB", t] <= model.x["AB"] * 3)
model.capacity_AC = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AC", t] <= model.x["AC"] * 6)
model.capacity_AF = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AF", t] <= model.x["AF"] * 1)
model.capacity_BC = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BC", t] <= model.x["BC"] * 3)
model.capacity_BD = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BD", t] <= model.x["BD"] * 10)
model.capacity_BE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BE", t] <= model.x["BE"] * 4)
model.capacity_CE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CE", t] <= model.x["CE"] * 4)
model.capacity_CF = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CF", t] <= model.x["CF"] * 4)
model.capacity_DE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["DE", t] <= model.x["DE"] * 5)
model.capacity_EG = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["EG", t] <= model.x["EG"] * 5)
model.capacity_FB = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FB", t] <= model.x["FB"] * 12)
model.capacity_FG = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FG", t] <= model.x["FG"] * 7)

# Solve the model
solver = pyo.SolverFactory('cbc') # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)

# Access the optimal values
print("Optimal Objective Value:", pyo.value(model.obj))

# Print pipe activation decisions
for i in model.Pipes:
    if pyo.value(model.x[i]) > 0.5:
        print(f"Pipe {i} is activated")

# You can access the flow and release values similarly.
```

In this code:

-  `f_data`: This dictionary represents the inflow at point A for each time step. **This is sample data.** You need to replace it with your actual inflow data. 
-  Comments explain each part of the code. 
-  You'll need to have Pyomo and a suitable solver (like CBC, GLPK, or others) installed to run this code. 


## 4. Input problem data and try running the generated code

In [262]:
import pyomo.environ as pyo

# Sample Data (You'll need to replace this with your actual data)
f_data = {1: 10, 2: 12, 3: 15, 4: 10, 5: 8, 6: 12, 7: 14, 8: 11, 9: 9, 10: 13} 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10) # Time periods
model.Pipes = pyo.Set(initialize=["AB", "AC", "AF", "BC", "BD", "BE", "CE", "CF", "DE", "EG", "FB", "FG"])
model.Nodes = pyo.Set(initialize=["B", "C", "D", "E", "F"])

# Variables
model.x = pyo.Var(model.Pipes, domain=pyo.Binary) # Pipe activation
model.f = pyo.Var(model.Pipes, model.T, domain=pyo.NonNegativeReals) # Flow in pipes
model.r = pyo.Var(model.Nodes, model.T, domain=pyo.NonNegativeReals) # Release at nodes

# Objective function
model.obj = pyo.Objective(expr=sum(model.f["EG", t] + model.f["FG", t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Pipe Activation Limit
model.pipe_activation = pyo.Constraint(expr=sum(model.x[i] for i in model.Pipes) == 10)

# 2. Water Flow at each Time Step
def flow_balance(model, t):
    if t==1:
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
    else: 
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
model.flow_A = pyo.Constraint(model.T, rule=flow_balance)

def flow_balance_B(model, t):
    return model.f["AB", t] + model.f["FB", t] == model.f["BC", t] + model.f["BD", t] + model.f["BE", t] + model.r["B", t]
model.flow_B = pyo.Constraint(model.T, rule=flow_balance_B)

def flow_balance_C(model, t):
    return model.f["AC", t] + model.f["BC", t] == model.f["CE", t] + model.f["CF", t] + model.r["C", t]
model.flow_C = pyo.Constraint(model.T, rule=flow_balance_C)

def flow_balance_D(model, t):
    return model.f["BD", t] + model.f["DE", t] == model.r["D", t]
model.flow_D = pyo.Constraint(model.T, rule=flow_balance_D)

def flow_balance_E(model, t):
    return model.f["BE", t] + model.f["CE", t] + model.f["DE", t] == model.f["EG", t] + model.r["E", t]
model.flow_E = pyo.Constraint(model.T, rule=flow_balance_E)

def flow_balance_F(model, t):
    return model.f["AF", t] + model.f["CF", t] + model.f["FB", t] == model.f["FG", t] + model.r["F", t]
model.flow_F = pyo.Constraint(model.T, rule=flow_balance_F)


# 3. Pipe Capacity Limits
model.capacity_AB = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AB", t] <= model.x["AB"] * 3)
model.capacity_AC = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AC", t] <= model.x["AC"] * 6)
model.capacity_AF = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AF", t] <= model.x["AF"] * 1)
model.capacity_BC = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BC", t] <= model.x["BC"] * 3)
model.capacity_BD = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BD", t] <= model.x["BD"] * 10)
model.capacity_BE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BE", t] <= model.x["BE"] * 4)
model.capacity_CE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CE", t] <= model.x["CE"] * 4)
model.capacity_CF = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CF", t] <= model.x["CF"] * 4)
model.capacity_DE = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["DE", t] <= model.x["DE"] * 5)
model.capacity_EG = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["EG", t] <= model.x["EG"] * 5)
model.capacity_FB = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FB", t] <= model.x["FB"] * 12)
model.capacity_FG = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FG", t] <= model.x["FG"] * 7)

# Solve the model
solver = pyo.SolverFactory('cbc') # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)

# Access the optimal values
print("Optimal Objective Value:", pyo.value(model.obj))

# Print pipe activation decisions
for i in model.Pipes:
    if pyo.value(model.x[i]) > 0.5:
        print(f"Pipe {i} is activated")

ERROR: Rule failed when generating expression for Constraint capacity_AB with
index 1: PyomoException: Cannot convert non-constant Pyomo expression (0  <=
f[AB,1]) to bool. This error is usually caused by using a Var, unit, or
mutable Param in a Boolean context such as an "if" statement, or when checking
container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'capacity_AB' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression (0  <=
        f[AB,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.

PyomoException: Cannot convert non-constant Pyomo expression (0  <=  f[AB,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [272]:
import pyomo.environ as pyo

# Sample Data (You'll need to replace this with your actual data)
f_data = {1: 10, 2: 12, 3: 15, 4: 10, 5: 8, 6: 12, 7: 14, 8: 11, 9: 9, 10: 13} 
def f(t):
    if t >= 0:
        return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)
    else:
        return 0

def populate_f_data(keys):
    f_data = {}
    for key in keys:
        f_data[key] = f(key)
    return f_data

f_data = populate_f_data(range(1,11))

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10) # Time periods
model.Pipes = pyo.Set(initialize=["AB", "AC", "AF", "BC", "BD", "BE", "CE", "CF", "DE", "EG", "FB", "FG"])
model.Nodes = pyo.Set(initialize=["B", "C", "D", "E", "F"])

# Variables
model.x = pyo.Var(model.Pipes, domain=pyo.Binary) # Pipe activation
model.f = pyo.Var(model.Pipes, model.T, domain=pyo.NonNegativeReals) # Flow in pipes
model.r = pyo.Var(model.Nodes, model.T, domain=pyo.NonNegativeReals) # Release at nodes

# Objective function
model.obj = pyo.Objective(expr=sum(model.f["EG", t] + model.f["FG", t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Pipe Activation Limit
model.pipe_activation = pyo.Constraint(expr=sum(model.x[i] for i in model.Pipes) == 10)

# 2. Water Flow at each Time Step
def flow_balance(model, t):
    if t==1:
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
    else: 
        return model.f["AB", t] + model.f["AC", t] + model.f["AF", t] == f_data[t]
model.flow_A = pyo.Constraint(model.T, rule=flow_balance)

def flow_balance_B(model, t):
    return model.f["AB", t] + model.f["FB", t] == model.f["BC", t] + model.f["BD", t] + model.f["BE", t] + model.r["B", t]
model.flow_B = pyo.Constraint(model.T, rule=flow_balance_B)

def flow_balance_C(model, t):
    return model.f["AC", t] + model.f["BC", t] == model.f["CE", t] + model.f["CF", t] + model.r["C", t]
model.flow_C = pyo.Constraint(model.T, rule=flow_balance_C)

def flow_balance_D(model, t):
    return model.f["BD", t] + model.f["DE", t] == model.r["D", t]
model.flow_D = pyo.Constraint(model.T, rule=flow_balance_D)

def flow_balance_E(model, t):
    return model.f["BE", t] + model.f["CE", t] + model.f["DE", t] == model.f["EG", t] + model.r["E", t]
model.flow_E = pyo.Constraint(model.T, rule=flow_balance_E)

def flow_balance_F(model, t):
    return model.f["AF", t] + model.f["CF", t] + model.f["FB", t] == model.f["FG", t] + model.r["F", t]
model.flow_F = pyo.Constraint(model.T, rule=flow_balance_F)


# 3. Pipe Capacity Limits
model.capacity_AB_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AB", t])
model.capacity_AB_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["AB", t] <= model.x["AB"] * 3)

model.capacity_AC_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AC", t])
model.capacity_AC_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["AC", t] <= model.x["AC"] * 6)

model.capacity_AF_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["AF", t])
model.capacity_AF_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["AF", t] <= model.x["AF"] * 1)

model.capacity_BC_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BC", t])
model.capacity_BC_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["BC", t] <= model.x["BC"] * 3)

model.capacity_BD_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BD", t])
model.capacity_BD_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["BD", t] <= model.x["BD"] * 10)

model.capacity_BE_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["BE", t])
model.capacity_BE_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["BE", t] <= model.x["BE"] * 4)

model.capacity_CE_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CE", t])
model.capacity_CE_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["CE", t] <= model.x["CE"] * 4)

model.capacity_CF_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["CF", t])
model.capacity_CF_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["CF", t] <= model.x["CF"] * 4)

model.capacity_DE_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["DE", t])
model.capacity_DE_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["DE", t] <= model.x["DE"] * 5)

model.capacity_EG_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["EG", t])
model.capacity_EG_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["EG", t] <= model.x["EG"] * 5)

model.capacity_FB_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FB", t])
model.capacity_FB_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["FB", t] <= model.x["FB"] * 12)

model.capacity_FG_lower = pyo.Constraint(model.T, rule=lambda model, t: 0 <= model.f["FG", t])
model.capacity_FG_upper = pyo.Constraint(model.T, rule=lambda model, t: model.f["FG", t] <= model.x["FG"] * 7)

# Solve the model
solver = pyo.SolverFactory('glpk') # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)

# Access the optimal values
print("Optimal Objective Value:", pyo.value(model.obj))

# Print pipe activation decisions
for i in model.Pipes:
    if pyo.value(model.x[i]) > 0.5:
        print(f"Pipe {i} is activated")


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 301
  Number of variables: 182
  Number of nonzeros: 642
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03961610794067383

ERROR: evaluating object as numeric value: f[EG,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object f[EG,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object f[EG,1]


ValueError: No value for uninitialized NumericValue object f[EG,1]

In [269]:
results = solver.solve(model, tee=True)

# Display solver output
results.write()

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpwcmbjqk_.glpk.raw
 --wglp C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpdc3u6ukq.glpk.glp
 --cpxlp C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpy891402r.pyomo.lp
Reading problem data from 'C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpy891402r.pyomo.lp'...
C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpy891402r.pyomo.lp:1757: warning: lower bound of variable 'x23' redefined
C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpy891402r.pyomo.lp:1757: warning: upper bound of variable 'x23' redefined
301 rows, 182 columns, 642 non-zeros
12 integer variables, all of which are binary
1769 lines were read
Writing problem data to 'C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpdc3u6ukq.glpk.glp'...
1569 lines were written
GLPK Integer Optimizer 5.0
301 rows, 182 columns, 642 non-zeros
12 integer variables, all of which are b

In [273]:
f_data

{1: 3.6326666666666667,
 2: 5.465333333333334,
 3: 7.698,
 4: 9.530666666666667,
 5: 10.163333333333332,
 6: 8.796,
 7: 4.628666666666666,
 8: 0,
 9: 0,
 10: 0}

## 6. Print the responses

In [274]:
print(response.text)

## Variables:

**1. Binary variables for pipe activation (1 if active, 0 if inactive):**

*  x_AB
*  x_AC
*  x_AF
*  x_BC
*  x_BD
*  x_BE
*  x_CE
*  x_CF
*  x_DE
*  x_EG
*  x_FB
*  x_FG

**2. Continuous variables for water flow through each pipe at each time step (t = 1, 2,..., 10):**

* f_AB(t)
* f_AC(t)
* f_AF(t)
* f_BC(t)
* f_BD(t)
* f_BE(t)
* f_CE(t)
* f_CF(t)
* f_DE(t)
* f_EG(t)
* f_FB(t)
* f_FG(t)

**3. Continuous variables for water released at each point at each time step (t=1, 2,..., 10):**

* r_B(t)
* r_C(t)
* r_D(t)
* r_E(t)
* r_F(t) 



In [275]:
print(response2.text)

Maximize:  ∑_{t=1}^{10} (f_EG(t) + f_FG(t)) 



In [276]:
print(response3.text)

## Constraints:

**1. Pipe Activation:**

*   **Limit on active pipes:**  ∑_{i∈{AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}} x_i = 10 

**2. Water Flow at each Time Step (t = 1, 2, ..., 10):**

*   **Inflow at A:**  f_AB(t) + f_AC(t) + f_AF(t)  = f(t) 
*   **Conservation of flow at B:** f_AB(t) + f_FB(t) = f_BC(t) + f_BD(t) + f_BE(t) + r_B(t)
*   **Conservation of flow at C:** f_AC(t) + f_BC(t) = f_CE(t) + f_CF(t) + r_C(t)
*   **Conservation of flow at D:** f_BD(t) + f_DE(t) = r_D(t) 
*   **Conservation of flow at E:** f_BE(t) + f_CE(t) + f_DE(t) = f_EG(t) + r_E(t)
*   **Conservation of flow at F:** f_AF(t) + f_CF(t) + f_FB(t) = f_FG(t) + r_F(t)

**3. Pipe Capacity:**

*   **Flow capacity limits:**
    *   0 ≤ f_AB(t) ≤ x_AB * 3 
    *   0 ≤ f_AC(t) ≤ x_AC * 6
    *   0 ≤ f_AF(t) ≤ x_AF * 1
    *   0 ≤ f_BC(t) ≤ x_BC * 3
    *   0 ≤ f_BD(t) ≤ x_BD * 10
    *   0 ≤ f_BE(t) ≤ x_BE * 4
    *   0 ≤ f_CE(t) ≤ x_CE * 4
    *   0 ≤ f_CF(t) ≤ x_CF * 4
    *   0 ≤ f_DE(t) ≤ x_DE * 5
    *  

In [277]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You'll need to replace this with your actual data)
f_data = {1: 10, 2: 12, 3: 15, 4: 10, 5: 8, 6: 12, 7: 14, 8: 11, 9: 9, 10: 13} 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10) # Time periods
model.Pipes = pyo.Set(initialize=["AB", "AC", "AF", "BC", "BD", "BE", "CE", "CF", "DE", "EG", "FB", "FG"])
model.Nodes = pyo.Set(initialize=["B", "C", "D", "E", "F"])

# Variables
model.x = pyo.Var(model.Pipes, domain=pyo.Binary) # Pipe activation
model.f = pyo.Var(model.Pipes, model.T, domain=pyo.NonNegativeReals) # Flow in pipes
model.r = pyo.Var(model.Nodes, model.T, domain=pyo.NonNegativeReals) # Release at nodes

# Objective function
model.obj = pyo.Objective(expr=sum(model.f["EG", t] + model.f["FG", t] for t in model.T), sense=pyo.maximize)

# Constraints
# 1. Pipe Activation Limit
model.pipe_activation = pyo.Constraint(expr=sum(model.x[i] for i in model.Pipes) == 10)

# 2. Water Flow at each Time Step
def flo